In [1]:
import os

In [2]:
%pwd

'/Users/kumara/Desktop/Machine_Learning_Projects/Age-Classification-Project/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kumara/Desktop/Machine_Learning_Projects/Age-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

    

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
    
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config= self.config.data_ingestion

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )

        return data_ingestion_config
    
    

    
    

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size


In [19]:
class DataIngestion:
    def __init__(self, data_ingestion_config: DataIngestionConfig):
        self.data_ingestion_config = data_ingestion_config

    def download_file(self):
        os.makedirs(os.path.dirname(self.data_ingestion_config.local_data_file), exist_ok=True)

        if not os.path.exists(self.data_ingestion_config.local_data_file):
            #print(self.data_ingestion_config.source_URL)
            #print(self.data_ingestion_config.local_data_file)
            filename,headers = request.urlretrieve(
                url=self.data_ingestion_config.source_URL,
                filename=self.data_ingestion_config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.data_ingestion_config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        """

        unzip_path = self.data_ingestion_config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.data_ingestion_config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Zip file extracted to: {unzip_path}")

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion( data_ingestion_config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-07-14 15:30:40,107: INFO: common: yaml file:config/config.yaml loaded successfully]
[2024-07-14 15:30:40,109: INFO: common: yaml file:params.yaml loaded successfully]
[2024-07-14 15:30:40,110: INFO: common: Directory created: artifacts]
[2024-07-14 15:30:46,062: INFO: 152677404: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 29011512
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "841be8abd24d5a63eccb2b9f3e366a7a487b7f0113959a9dd326c69fa0160646"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 3923:2AF5B6:1B2918:21A8F6:66939C84
Accept-Ranges: bytes
Date: Sun, 14 Jul 2024 10:00:41 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg120088-QPG
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1720951241.782834,VS0,VE449
Vary: Authorization,Accept-Encoding,